### Clasificación de imágenes multiespectro con CNN

In [99]:
import numpy as np
import random
import spectral
import scipy
import scipy.ndimage
from scipy.ndimage import rotate
import scipy.io as sio


from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from tensorflow.keras import utils as np_utils
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import SGD
from keras import backend as K

K.set_image_data_format('channels_last')

In [100]:
# Cargar las imágenes y las etiquetas
data = sio.loadmat('Data\\salinas_corrected.mat')['salinas_corrected']
labels = sio.loadmat('Data\\salinas_gt.mat')['salinas_gt']

# Mostrar cuantas veces aparece cada etiqueta
uniqueLabels, labelCounts = np.unique(labels, return_counts=True)
for l, c in zip(uniqueLabels, labelCounts):
    print(f'{l} --> {c}')

0 --> 56975
1 --> 2009
2 --> 3726
3 --> 1976
4 --> 1394
5 --> 2678
6 --> 3959
7 --> 3579
8 --> 11271
9 --> 6203
10 --> 3278
11 --> 1068
12 --> 1927
13 --> 916
14 --> 1070
15 --> 7268
16 --> 1807


In [101]:
# Aplicar PCA
new_data = np.reshape(data, (-1, data.shape[2])) # Reshapea de 3D (ancho, alto, bandas) a 2D (pixels, bandas): (512, 217, 204) --> (111104, 204)
pca = PCA(n_components=30, whiten=True) # Selecciona los componentes principales y los normaliza
new_data = pca.fit_transform(new_data)
new_data = np.reshape(new_data, (data.shape[0], data.shape[1], 30))

In [102]:
# Crea un borde de ceros, se usa en la siguiente celda para crear parches
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [103]:
# Crear parches (regiones cuadradas alrededor de cada pixel)
windowSize = 1
margin = int((windowSize - 1) / 2) # Pixeles a incluir alrededor de cada pixel parcheado
zeroPaddedX = padWithZeros(new_data, margin=margin) # Crea un borde de ceros alrededor de la imagen para que todos los pixeles tengan parches completos
patchesData = np.zeros((new_data.shape[0] * new_data.shape[1], windowSize, windowSize, new_data.shape[2])) # Array para almacenar datos de los parches (1, 1, 204 [bandas])
patchesLabels = np.zeros((new_data.shape[0] * new_data.shape[1])) # Array para almacenar la etiqueta de cada parche
patchIndex = 0 # Inicia un índice para cada parche
removeZeroLabels = True # Bandera para eliminar los parches con etiqueta 0

# Obtener el parche de cada pixel
for r in range(margin, zeroPaddedX.shape[0] - margin): # Índice de la fila, recorre toda la imagen evitando el padding (de margin=0 a altura-margin=altura completa)
    for c in range(margin, zeroPaddedX.shape[1] - margin): # Índice de la columna, haciendo que cada pixel tenga las coordenadas (r, c)
        patch = zeroPaddedX[r - margin:r + margin + 1, c-margin:c + margin + 1] # Selecciona los pixels de alrededor del pixel (parche)
        patchesData[patchIndex, :, :, :] = patch # Guarda los datos del parche
        patchesLabels[patchIndex] = labels[r-margin, c-margin] # Guarda la etiqueta de ese parche siguiendo el mismo índice
        patchIndex += 1

# Eliminar los parches etiquetados como 0 (no tienen info o son áreas de fondo no útiles para el entrenamiento)
if removeZeroLabels:
    patchesData = patchesData[patchesLabels > 0, :, :, :]
    patchesLabels = patchesLabels[patchesLabels > 0]
    patchesLabels -= 1

In [104]:
print(f'Forma del array de parches:')
print(f'Parches: {patchesData.shape[0]}\t Altura: {patchesData.shape[1]}\t Anchura: {patchesData.shape[2]}\t Bandas espectrales: {patchesData.shape[3]}')

Forma del array de parches:
Parches: 54129	 Altura: 1	 Anchura: 1	 Bandas espectrales: 30


In [105]:
# Crear los conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(patchesData, patchesLabels, train_size=0.75, stratify=patchesLabels, random_state=11)

In [106]:
# Sobremuestreo de las clases minoritarias para equilibrar las apariciones
uniqueLabels, labelCounts = np.unique(y_train, return_counts=True)
maxCount = np.max(labelCounts) # Apariciones de la clase mayoritaria
labelInverseRatios = maxCount / labelCounts # Array que invierte el peso de las apariciones de cada clase
X_train = X_train[y_train == uniqueLabels[0], :].repeat(round(labelInverseRatios[0]), axis=0) # Multiplica las muestras de la primera clase por el ratio invertido
y_train = y_train[y_train == uniqueLabels[0]].repeat(round(labelInverseRatios[0]), axis=0) # Misma operación para mantener la relación entre X e y

# Recorrer las demás clases aplicando las dos operaciones anteriores (apariciones * ratio inverso)
for label, labelInverseRatio in zip(uniqueLabels[1:], labelInverseRatios[1:]):
    cX = X_train[y_train == label, :].repeat(round(labelInverseRatio), axis=0)
    cY = y_train[y_train == label].repeat(round(labelInverseRatio), axis=0)

    # Ir añadiendo las clases ya equilibradas
    X_train = np.concatenate((X_train, cX))
    y_train = np.concatenate((y_train, cY))

# Permutar los datos para que las clases no aparezcan seguidas (ej: [1, 1, 1, 1, 2, 2, 2, 2...] --> [1, 2, 2, 4, 6, 1, 3, 4...])
np.random.seed(seed=42)
random_permutation = np.random.permutation(y_train.shape[0]) # Genera una lista aleatoria de índices
X_train = X_train[random_permutation, :] # Reorganiza el array con el nuevo orden aleatorio de índices
y_train = y_train[random_permutation] # Lo mismo

In [107]:
# Hacer data augmentation a la mitad de los parches
for i in range(int(X_train.shape[0]/2)):
    patch = X_train[i, :, :, :] # Selecciona el parche con todos sus datos (alto, ancho, bandas)
    num = random.randint(0, 2) # Genera un número aleatorio (0, 1 o 2)
    if num == 0: # Volteo vertical
        flipped_patch = np.flipud(patch)
    elif num == 1:  # Volteo horizontal (izq a dcha)
        flipped_patch = np.fliplr(patch)
    else: # Rotación aleatoria
        no = random.randrange(-180,180,30)
        flipped_patch = scipy.ndimage.rotate(patch, no,axes=(1, 0),reshape=False, output=None, order=3, mode='constant', cval=0.0, prefilter=False)
    
    # Asignar el nuevo parche augmentado
    patch2 = flipped_patch
    X_train[i, :, :, :] = patch2